In [59]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [60]:
! dvc pull

/1 [00:00<?,     ?file/s]
!
  0%|          |data/prepared                       0/137 [00:00<?,       ?it/s]
100%|██████████|data/prepared                 137/137 [00:00<00:00,      356B/s]
  0% Downloading|                                    |0/1 [00:00<?,     ?file/s]
!
  0%|          |data/features                       0/137 [00:00<?,       ?it/s]
100%|██████████|data/features                 137/137 [00:00<00:00,      378B/s]
  0% Downloading|                                    |0/6 [00:00<?,     ?file/s]
!
  0%|          |data/prepared/test.tsv         0.00/4.76M [00:00<?,       ?it/s]

!

  0%|          |data/prepared/train.tsv        0.00/19.1M [00:00<?,       ?it/s]


!


  0%|          |model.pkl                      0.00/2.66M [00:00<?,       ?it/s]



!



  0%|          |data/features/test.pkl         0.00/1.41M [00:00<?,       ?it/s]




!




  0%|          |data/data.xml                  0.00/37.9M [00:00<?,       ?it/s]





!





  0%|          |data/features/train.p

## Easily reproduce somewhere else

### from cache

In [61]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [62]:
! dvc repro -f

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml
  0% Saving prepared|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>
Running s

In [63]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

### Assert equivalent result

In [64]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [65]:
! dvc status

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Data and pipelines are up to date.


In [66]:
! dvc params diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [67]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [68]:
! dvc metrics diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [69]:
! cat scores.json

{"auc": 0.5417487597055675}

In [70]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html


## For reproducibility of this notebook:
### Remove tracked data from dvc (local and remote) and
### Remove unnecessary files 

In [71]:
! dvc remove data/.

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>ERROR: failed to remove 'data/.' - "Stage 'data/.' not found inside 'dvc.yaml' file"

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [73]:
! dvc remove data/data.xml.dvc
! dvc remove data/features/train.pkl
! dvc remove data/features/test.pkl
! dvc remove data/prepared/train.tsv
! dvc remove data/prepared/test.tsv
! dvc remove model.pkl

ERROR: failed to remove 'data/data.xml.dvc' - 'data/data.xml.dvc' does not exist.

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
ERROR: failed to remove 'data/features/train.pkl' - "Stage 'data/features/train.pkl' not found inside 'dvc.yaml' file"

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>ERROR: failed to remove 'data/features/test.pkl' - "Stage 'data/features/test.pkl' not found inside 'dvc.yaml' file"

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>ERROR: failed to remove 'data/prepared/train.tsv' - "Stage 'data/prepared/train.tsv' not found inside 'dvc.yaml' file"

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
ERROR: failed to remove 'data/prepared/test.tsv' - "Stage 'data/pre

In [74]:
! rm -rf data/
! rm plots.html
! rm prc.json
! rm model.pkl
! rm data/data.xml

rm: cannot remove 'data/data.xml': No such file or directory


In [75]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Deleted:
    data/data.xml
    data/features/
    data/prepared/
    model.pkl
    prc.json

files summary: 0 added, 3 deleted, 0 modified


In [76]:
! git add data/data.xml.dvc
! git commit -m 'removes data'
! git push origin tutorial
! dvc push

[data_pipelines a18c8ed] removes data
 1 file changed, 3 deletions(-)
 delete mode 100644 data/data.xml.dvc
Everything up-to-date
Everything is up to date.


In [77]:
! echo "y" | dvc gc -cw

  0% Querying cache in 'gs://dvc_intro'|  |1.00/4.10k [00:00<10:45,  6.34file/s]

In [78]:
! echo 'y' | dvc destroy

This will destroy all information about your pipelines, all data files, as well as cache in .dvc/cache.
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

In [79]:
! git add .
! git commit -m "clean"
! git push origin tutorial

[data_pipelines a7e37f2] clean
 10 files changed, 235 deletions(-)
 delete mode 100644 .dvc/.gitignore
 delete mode 100644 .dvc/config
 delete mode 100644 .dvc/plots/confusion.json
 delete mode 100644 .dvc/plots/default.json
 delete mode 100644 .dvc/plots/scatter.json
 delete mode 100644 .dvc/plots/smooth.json
 delete mode 100644 data/.gitignore
 delete mode 100644 dvc.lock
 delete mode 100644 dvc.yaml
 delete mode 100644 prc.json
Everything up-to-date
